# Task 3: Correlation Analysis - News Sentiment vs Stock Returns

**Objective**: Establish quantifiable correlations between financial news sentiment and stock price movements to develop actionable investment insights for Nova Financial Solutions.

## Business Context

This analysis addresses the fundamental question: **Can news sentiment predict stock price movements?** By correlating sentiment polarity scores from 1.4M+ financial news headlines with daily stock returns across 6 major technology companies, we aim to:

1. **Validate Predictive Power**: Determine if sentiment has statistically significant correlation with returns
2. **Identify Lag Effects**: Test if today's sentiment predicts tomorrow's price movements (T+1, T+2)
3. **Quantify Relationships**: Calculate Pearson correlation coefficients and p-values for confidence assessment
4. **Generate Trading Signals**: Develop actionable recommendations based on sentiment-return patterns

**Expected Outcomes**:
- Correlation coefficients (target: >0.3 for meaningful relationships)
- Statistical significance (p-value < 0.05 for confidence)
- Predictive lag analysis for algorithmic trading strategies
- Risk-adjusted investment recommendations

---

## 1. Import Required Libraries

Import core data processing, NLP, statistical analysis, and custom modules for sentiment-price correlation analysis.

In [ ]:
# Standard library imports
import sys
import warnings
from pathlib import Path

# Data manipulation
import pandas as pd
import numpy as np

# Statistical analysis
from scipy import stats
from scipy.stats import pearsonr

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# NLP libraries
import nltk
from textblob import TextBlob

# Configure plotting
warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")

# Add src to path for custom modules
sys.path.append(str(Path().absolute().parent))

# Import custom modules
from src.core.data_loader import DataLoader
from src.core.data_processor import DataProcessor
from src.core.sentiment_analyzer import SentimentAnalyzer
from src.core.financial_analyzer import FinancialAnalyzer
from src.core.visualizer import Visualizer

print("✅ All libraries imported successfully!")
print(f"📊 pandas version: {pd.__version__}")
print(f"🔢 numpy version: {np.__version__}")

✅ All libraries imported successfully!
📊 pandas version: 2.3.3
🔢 numpy version: 2.2.6


## 2. Load News and Stock Data

Load 1.4M+ financial news articles and historical stock price data (OHLCV) for correlation analysis.

In [3]:
# Initialize data loader
loader = DataLoader()

# Load news data
print("📰 Loading financial news data...")
news_df = loader.load_news_data("../data/raw/raw_analyst_ratings.csv")
print(f"✅ Loaded {len(news_df):,} news articles")
print(f"   Date range: {news_df['date'].min()} to {news_df['date'].max()}")
print(f"   Unique stocks: {news_df['stock'].nunique()}")
print(f"   Unique publishers: {news_df['publisher'].nunique()}")

# Load stock data for all 6 symbols
stock_symbols = ["AAPL", "AMZN", "GOOG", "META", "MSFT", "NVDA"]
stock_data = {}

print(f"\n📈 Loading stock price data for {len(stock_symbols)} symbols...")
for symbol in stock_symbols:
    try:
        stock_df = loader.load_stock_data(f"../data/cleaned/{symbol}.csv")
        stock_data[symbol] = stock_df
        print(f"   ✅ {symbol}: {len(stock_df)} trading days")
    except Exception as e:
        print(f"   ❌ {symbol}: Error - {e}")

print(f"\n✅ Successfully loaded {len(stock_data)} stock datasets")

📰 Loading financial news data...
✅ Loaded 1,407,328 news articles
✅ Loaded 1,407,328 news articles
   Date range: 2009-02-14 00:00:00 to 2020-06-11 17:12:35-04:00
   Unique stocks: 6204
   Date range: 2009-02-14 00:00:00 to 2020-06-11 17:12:35-04:00
   Unique stocks: 6204
   Unique publishers: 1034

📈 Loading stock price data for 6 symbols...
   Unique publishers: 1034

📈 Loading stock price data for 6 symbols...
   ✅ AAPL: 3774 trading days
   ✅ AMZN: 3774 trading days
   ✅ GOOG: 3774 trading days
   ✅ META: 2923 trading days
   ✅ MSFT: 3774 trading days
   ✅ NVDA: 3774 trading days

✅ Successfully loaded 6 stock datasets
   ✅ AAPL: 3774 trading days
   ✅ AMZN: 3774 trading days
   ✅ GOOG: 3774 trading days
   ✅ META: 2923 trading days
   ✅ MSFT: 3774 trading days
   ✅ NVDA: 3774 trading days

✅ Successfully loaded 6 stock datasets


## 3. Date Normalization and Alignment

Convert UTC-4 news timestamps to trading days, mapping weekend/holiday news to the next trading day (Monday) for proper alignment with stock data.

In [8]:
# Reload modules to pick up fixes
import importlib
import src.core.data_processor

importlib.reload(src.core.data_processor)
from src.core.data_processor import DataProcessor

processor = DataProcessor()
print("✅ Module reloaded")

✅ Module reloaded


In [9]:
# Initialize data processor
processor = DataProcessor()

# Normalize news dates to trading days
print("🕐 Normalizing news dates to trading days...")
news_normalized = processor.normalize_dates(news_df, date_column="date")

# Display normalization results
print(f"✅ News dates normalized")
print(f"   Original date range: {news_df['date'].min()} to {news_df['date'].max()}")
print(
    f"   Trading date range: {news_normalized['trading_date'].min()} to {news_normalized['trading_date'].max()}"
)
print(
    f"   Weekend articles remapped: {(news_normalized['trading_date'].dt.dayofweek >= 5).sum():,}"
)

# Check for duplicate dates (multiple articles same day/stock)
duplicates = news_normalized.groupby(["trading_date", "stock"]).size()
print(f"   Articles per day (avg): {duplicates.mean():.2f}")
print(f"   Max articles same day/stock: {duplicates.max()}")

# Preview normalized data
news_normalized[["headline", "stock", "trading_date"]].head(10)

🕐 Normalizing news dates to trading days...


AttributeError: Can only use .dt accessor with datetimelike values

## 4. Sentiment Analysis on Headlines

Apply TextBlob sentiment analysis to extract polarity scores (-1 to +1) from news headlines, classifying sentiment as positive, negative, or neutral.

In [ ]:
# Initialize sentiment analyzer
sentiment_analyzer = SentimentAnalyzer()

# Analyze all headlines for sentiment
print("💭 Analyzing sentiment for all headlines...")
print(f"   Total headlines to process: {len(news_normalized):,}")

# Batch sentiment analysis (efficient for large datasets)
news_with_sentiment = sentiment_analyzer.analyze_dataframe(
    news_normalized, text_column="headline"
)

# Display sentiment distribution
print(f"\n✅ Sentiment analysis complete!")
print(f"\n📊 Sentiment Distribution:")
print(news_with_sentiment["sentiment_category"].value_counts())

sentiment_stats = news_with_sentiment["sentiment_polarity"].describe()
print(f"\n📈 Sentiment Polarity Statistics:")
print(f"   Mean: {sentiment_stats['mean']:.4f}")
print(f"   Median: {sentiment_stats['50%']:.4f}")
print(f"   Std Dev: {sentiment_stats['std']:.4f}")
print(f"   Range: [{sentiment_stats['min']:.4f}, {sentiment_stats['max']:.4f}]")

# Preview sentiment scores
news_with_sentiment[
    ["headline", "stock", "sentiment_polarity", "sentiment_category"]
].head(10)

### Visualize Sentiment Distribution

Examine the distribution of sentiment scores across all headlines to understand the sentiment landscape.

In [ ]:
# Create sentiment distribution visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Histogram of sentiment polarity scores
axes[0].hist(
    news_with_sentiment["sentiment_polarity"], bins=50, edgecolor="black", alpha=0.7
)
axes[0].axvline(x=0, color="red", linestyle="--", linewidth=2, label="Neutral (0)")
axes[0].set_xlabel("Sentiment Polarity Score", fontsize=12)
axes[0].set_ylabel("Frequency", fontsize=12)
axes[0].set_title(
    "Distribution of Sentiment Polarity Scores\n(All Headlines)",
    fontsize=14,
    fontweight="bold",
)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Bar chart of sentiment categories
category_counts = news_with_sentiment["sentiment_category"].value_counts()
colors = {"positive": "green", "negative": "red", "neutral": "gray"}
category_colors = [colors.get(cat, "blue") for cat in category_counts.index]

axes[1].bar(
    category_counts.index,
    category_counts.values,
    color=category_colors,
    edgecolor="black",
    alpha=0.7,
)
axes[1].set_xlabel("Sentiment Category", fontsize=12)
axes[1].set_ylabel("Number of Headlines", fontsize=12)
axes[1].set_title(
    "Sentiment Classification Distribution\n(Positive/Neutral/Negative)",
    fontsize=14,
    fontweight="bold",
)
axes[1].grid(True, alpha=0.3, axis="y")

# Add count labels on bars
for i, (cat, count) in enumerate(category_counts.items()):
    axes[1].text(
        i,
        count + len(news_with_sentiment) * 0.01,
        f"{count:,}",
        ha="center",
        va="bottom",
        fontsize=10,
        fontweight="bold",
    )

plt.tight_layout()
plt.show()

print(f"📊 Sentiment Distribution Insights:")
print(
    f"   Positive: {(news_with_sentiment['sentiment_category'] == 'positive').sum():,} ({(news_with_sentiment['sentiment_category'] == 'positive').sum()/len(news_with_sentiment)*100:.1f}%)"
)
print(
    f"   Neutral:  {(news_with_sentiment['sentiment_category'] == 'neutral').sum():,} ({(news_with_sentiment['sentiment_category'] == 'neutral').sum()/len(news_with_sentiment)*100:.1f}%)"
)
print(
    f"   Negative: {(news_with_sentiment['sentiment_category'] == 'negative').sum():,} ({(news_with_sentiment['sentiment_category'] == 'negative').sum()/len(news_with_sentiment)*100:.1f}%)"
)

## 5. Calculate Daily Stock Returns

Compute daily percentage returns for each stock using the formula: `(Close_today - Close_yesterday) / Close_yesterday * 100`.

In [ ]:
# Initialize financial analyzer
financial_analyzer = FinancialAnalyzer()

# Calculate returns for all stocks
print("📈 Calculating daily returns for all stocks...")
stock_returns = {}

for symbol, stock_df in stock_data.items():
    # Calculate returns
    returns_df = financial_analyzer.calculate_returns(stock_df.copy())

    # Add symbol column for later merging
    returns_df["stock"] = symbol

    # Rename Date to trading_date for alignment
    if "Date" in returns_df.columns:
        returns_df["trading_date"] = pd.to_datetime(returns_df["Date"])
    else:
        returns_df["trading_date"] = pd.to_datetime(returns_df.index)

    stock_returns[symbol] = returns_df

    # Display statistics
    returns_stats = returns_df["daily_return"].describe()
    print(f"\n✅ {symbol}:")
    print(f"   Mean return: {returns_stats['mean']:.4f}%")
    print(f"   Volatility (std): {returns_stats['std']:.4f}%")
    print(f"   Range: [{returns_stats['min']:.4f}%, {returns_stats['max']:.4f}%]")

# Combine all returns into single DataFrame
all_returns = pd.concat(
    [
        df[["trading_date", "stock", "daily_return", "Close"]]
        for df in stock_returns.values()
    ],
    ignore_index=True,
)

print(f"\n📊 Combined Returns Dataset:")
print(f"   Total observations: {len(all_returns):,}")
print(
    f"   Date range: {all_returns['trading_date'].min()} to {all_returns['trading_date'].max()}"
)
print(f"   Stocks: {all_returns['stock'].unique().tolist()}")

## 6. Aggregate Daily Sentiment Scores

Group sentiment scores by date and stock symbol to calculate daily average sentiment, handling multiple news articles per day.

In [ ]:
# Aggregate daily sentiment by stock
print("📊 Aggregating daily sentiment scores...")
daily_sentiment = sentiment_analyzer.aggregate_daily_sentiment(
    news_with_sentiment, date_column="trading_date"
)

# Display aggregation statistics
print(f"\n✅ Daily sentiment aggregated")
print(f"   Total trading days with news: {len(daily_sentiment):,}")
print(
    f"   Date range: {daily_sentiment['trading_date'].min()} to {daily_sentiment['trading_date'].max()}"
)
print(f"   Stocks with daily sentiment: {daily_sentiment['stock'].nunique()}")

# Check aggregation statistics
print(f"\n📈 Aggregation Statistics:")
print(f"   Average articles per day: {daily_sentiment['article_count'].mean():.2f}")
print(f"   Max articles same day: {daily_sentiment['article_count'].max()}")
print(f"   Days with single article: {(daily_sentiment['article_count'] == 1).sum():,}")
print(f"   Days with 5+ articles: {(daily_sentiment['article_count'] >= 5).sum():,}")

# Display sample of aggregated data
print(f"\n📋 Sample Daily Sentiment Data:")
daily_sentiment[
    ["trading_date", "stock", "avg_sentiment", "article_count", "positive_ratio"]
].head(10)

## 7. Merge Sentiment and Returns Data

Align daily sentiment scores with stock returns by date and symbol to create the unified dataset for correlation analysis.

In [ ]:
# Merge sentiment and returns on trading_date and stock
print("🔗 Merging sentiment and returns data...")
merged_data = pd.merge(
    daily_sentiment, all_returns, on=["trading_date", "stock"], how="inner"
)

# Display merge results
print(f"\n✅ Data merged successfully")
print(f"   Total matched observations: {len(merged_data):,}")
print(
    f"   Date range: {merged_data['trading_date'].min()} to {merged_data['trading_date'].max()}"
)
print(f"   Stocks in merged data: {merged_data['stock'].nunique()}")

# Check for missing values
missing_sentiment = merged_data["avg_sentiment"].isna().sum()
missing_returns = merged_data["daily_return"].isna().sum()
print(f"\n🔍 Data Quality:")
print(f"   Missing sentiment scores: {missing_sentiment}")
print(f"   Missing return values: {missing_returns}")

# Distribution by stock
print(f"\n📊 Observations per Stock:")
for symbol in merged_data["stock"].unique():
    count = len(merged_data[merged_data["stock"] == symbol])
    print(f"   {symbol}: {count:,} days")

# Preview merged data
print(f"\n📋 Sample Merged Data:")
merged_data[
    ["trading_date", "stock", "avg_sentiment", "daily_return", "article_count"]
].head(10)

## 8. Correlation Analysis: Sentiment vs Returns

Calculate Pearson correlation coefficients between daily sentiment scores and daily stock returns, with statistical significance testing.

In [ ]:
# Calculate overall correlation (all stocks combined)
print("📈 Calculating Sentiment-Return Correlations...\n")

# Remove any NaN values for correlation calculation
clean_data = merged_data.dropna(subset=["avg_sentiment", "daily_return"])

# Overall correlation
overall_corr, overall_pval = pearsonr(
    clean_data["avg_sentiment"], clean_data["daily_return"]
)

print(f"🌐 Overall Correlation (All Stocks Combined):")
print(f"   Pearson Correlation: {overall_corr:.4f}")
print(f"   P-value: {overall_pval:.6f}")
print(
    f"   Significance: {'✅ Statistically Significant (p < 0.05)' if overall_pval < 0.05 else '❌ Not Significant (p >= 0.05)'}"
)
print(f"   Observations: {len(clean_data):,}")

# Per-stock correlations
print(f"\n📊 Per-Stock Correlations:")
stock_correlations = []

for symbol in sorted(clean_data["stock"].unique()):
    stock_data = clean_data[clean_data["stock"] == symbol]

    if len(stock_data) > 2:  # Need at least 3 points for correlation
        corr, pval = pearsonr(stock_data["avg_sentiment"], stock_data["daily_return"])

        stock_correlations.append(
            {
                "stock": symbol,
                "correlation": corr,
                "p_value": pval,
                "significant": pval < 0.05,
                "observations": len(stock_data),
            }
        )

        sig_icon = "✅" if pval < 0.05 else "❌"
        print(
            f"   {symbol}: r={corr:+.4f}, p={pval:.6f} {sig_icon} (n={len(stock_data):,})"
        )

# Create DataFrame for correlations
correlations_df = pd.DataFrame(stock_correlations)

# Statistical summary
print(f"\n📈 Correlation Statistics:")
print(f"   Mean correlation: {correlations_df['correlation'].mean():.4f}")
print(f"   Median correlation: {correlations_df['correlation'].median():.4f}")
print(f"   Std deviation: {correlations_df['correlation'].std():.4f}")
print(
    f"   Range: [{correlations_df['correlation'].min():.4f}, {correlations_df['correlation'].max():.4f}]"
)
print(
    f"   Stocks with significant correlation: {correlations_df['significant'].sum()} / {len(correlations_df)}"
)

## 9. Lagged Correlation Analysis

Test if today's sentiment predicts tomorrow's returns (lag-1) and day-after returns (lag-2) to identify predictive patterns.

In [ ]:
# Calculate lagged correlations (sentiment today vs returns tomorrow/day-after)
print("⏰ Calculating Lagged Correlations...\n")

# Prepare data with lags by stock
lagged_correlations = []

for symbol in sorted(clean_data["stock"].unique()):
    stock_data = clean_data[clean_data["stock"] == symbol].sort_values("trading_date")

    if len(stock_data) < 5:  # Need enough data for lagged analysis
        continue

    # Create lagged returns (shift forward)
    stock_data = stock_data.copy()
    stock_data["return_t1"] = stock_data["daily_return"].shift(-1)  # Tomorrow's return
    stock_data["return_t2"] = stock_data["daily_return"].shift(-2)  # Day-after return

    # Same-day correlation (T+0)
    valid_t0 = stock_data.dropna(subset=["avg_sentiment", "daily_return"])
    if len(valid_t0) > 2:
        corr_t0, pval_t0 = pearsonr(valid_t0["avg_sentiment"], valid_t0["daily_return"])
    else:
        corr_t0, pval_t0 = np.nan, np.nan

    # Lag-1 correlation (T+1): sentiment today vs return tomorrow
    valid_t1 = stock_data.dropna(subset=["avg_sentiment", "return_t1"])
    if len(valid_t1) > 2:
        corr_t1, pval_t1 = pearsonr(valid_t1["avg_sentiment"], valid_t1["return_t1"])
    else:
        corr_t1, pval_t1 = np.nan, np.nan

    # Lag-2 correlation (T+2): sentiment today vs return day-after
    valid_t2 = stock_data.dropna(subset=["avg_sentiment", "return_t2"])
    if len(valid_t2) > 2:
        corr_t2, pval_t2 = pearsonr(valid_t2["avg_sentiment"], valid_t2["return_t2"])
    else:
        corr_t2, pval_t2 = np.nan, np.nan

    lagged_correlations.append(
        {
            "stock": symbol,
            "corr_t0": corr_t0,
            "pval_t0": pval_t0,
            "corr_t1": corr_t1,
            "pval_t1": pval_t1,
            "corr_t2": corr_t2,
            "pval_t2": pval_t2,
            "observations": len(stock_data),
        }
    )

    print(f"{symbol}:")
    print(
        f"   T+0 (same day):  r={corr_t0:+.4f}, p={pval_t0:.6f} {'✅' if pval_t0 < 0.05 else '❌'}"
    )
    print(
        f"   T+1 (next day):  r={corr_t1:+.4f}, p={pval_t1:.6f} {'✅' if pval_t1 < 0.05 else '❌'}"
    )
    print(
        f"   T+2 (2 days out): r={corr_t2:+.4f}, p={pval_t2:.6f} {'✅' if pval_t2 < 0.05 else '❌'}"
    )
    print()

# Create DataFrame
lagged_df = pd.DataFrame(lagged_correlations)

# Summary statistics
print(f"📊 Lagged Correlation Summary:")
print(f"   Mean T+0 correlation: {lagged_df['corr_t0'].mean():.4f}")
print(f"   Mean T+1 correlation: {lagged_df['corr_t1'].mean():.4f}")
print(f"   Mean T+2 correlation: {lagged_df['corr_t2'].mean():.4f}")
print(f"\n   Significant T+0: {(lagged_df['pval_t0'] < 0.05).sum()} / {len(lagged_df)}")
print(f"   Significant T+1: {(lagged_df['pval_t1'] < 0.05).sum()} / {len(lagged_df)}")
print(f"   Significant T+2: {(lagged_df['pval_t2'] < 0.05).sum()} / {len(lagged_df)}")

## 10. Visualize Correlation Results

Create comprehensive visualizations showing sentiment-return relationships: scatter plots, correlation heatmaps, and time series comparisons.

In [ ]:
# 1. Scatter Plot: Sentiment vs Returns (All Stocks)
fig, ax = plt.subplots(figsize=(12, 7))

for symbol in sorted(clean_data["stock"].unique()):
    stock_data = clean_data[clean_data["stock"] == symbol]
    ax.scatter(
        stock_data["avg_sentiment"],
        stock_data["daily_return"],
        alpha=0.5,
        s=30,
        label=symbol,
    )

# Add regression line
z = np.polyfit(clean_data["avg_sentiment"], clean_data["daily_return"], 1)
p = np.poly1d(z)
x_line = np.linspace(
    clean_data["avg_sentiment"].min(), clean_data["avg_sentiment"].max(), 100
)
ax.plot(
    x_line,
    p(x_line),
    "r--",
    linewidth=2,
    alpha=0.8,
    label=f"Trend Line (r={overall_corr:.3f})",
)

ax.axhline(y=0, color="black", linestyle="-", linewidth=0.5, alpha=0.5)
ax.axvline(x=0, color="black", linestyle="-", linewidth=0.5, alpha=0.5)

ax.set_xlabel("Daily Average Sentiment Score", fontsize=13, fontweight="bold")
ax.set_ylabel("Daily Stock Return (%)", fontsize=13, fontweight="bold")
ax.set_title(
    f"Sentiment vs Stock Returns Correlation\nPearson r={overall_corr:.4f}, p={overall_pval:.6f}",
    fontsize=15,
    fontweight="bold",
)
ax.legend(loc="best", fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 2. Correlation Heatmap: Per-Stock Correlations
fig, ax = plt.subplots(figsize=(10, 6))

# Prepare data for heatmap
heatmap_data = correlations_df.set_index("stock")[["correlation"]].T

# Create heatmap
sns.heatmap(
    heatmap_data,
    annot=True,
    fmt=".4f",
    cmap="RdYlGn",
    center=0,
    vmin=-0.5,
    vmax=0.5,
    cbar_kws={"label": "Pearson Correlation"},
    linewidths=2,
    linecolor="black",
    ax=ax,
)

ax.set_title(
    "Sentiment-Return Correlation by Stock\n(Darker Green = Stronger Positive Correlation)",
    fontsize=14,
    fontweight="bold",
)
ax.set_xlabel("")
ax.set_ylabel("")

plt.tight_layout()
plt.show()

print("📊 Correlation Heatmap Interpretation:")
print("   Green: Positive correlation (sentiment ↑ → returns ↑)")
print("   Red: Negative correlation (sentiment ↑ → returns ↓)")
print("   Yellow: Weak/No correlation")

In [ ]:
# 3. Lagged Correlation Comparison
fig, ax = plt.subplots(figsize=(12, 7))

x = np.arange(len(lagged_df))
width = 0.25

ax.bar(
    x - width,
    lagged_df["corr_t0"],
    width,
    label="T+0 (Same Day)",
    alpha=0.8,
    color="blue",
)
ax.bar(x, lagged_df["corr_t1"], width, label="T+1 (Next Day)", alpha=0.8, color="green")
ax.bar(
    x + width,
    lagged_df["corr_t2"],
    width,
    label="T+2 (2 Days Out)",
    alpha=0.8,
    color="orange",
)

ax.axhline(y=0, color="black", linestyle="-", linewidth=1)
ax.set_xlabel("Stock Symbol", fontsize=13, fontweight="bold")
ax.set_ylabel("Pearson Correlation Coefficient", fontsize=13, fontweight="bold")
ax.set_title(
    "Lagged Sentiment-Return Correlations\n(Testing Predictive Power)",
    fontsize=15,
    fontweight="bold",
)
ax.set_xticks(x)
ax.set_xticklabels(lagged_df["stock"])
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

print("📊 Lagged Correlation Insights:")
print("   T+0: Sentiment and returns on same day (contemporaneous)")
print("   T+1: Today's sentiment predicting tomorrow's return (1-day lag)")
print("   T+2: Today's sentiment predicting day-after return (2-day lag)")

In [ ]:
# 4. Time Series: Sentiment and Returns Over Time (Example: AAPL)
example_stock = "AAPL"
stock_ts = (
    clean_data[clean_data["stock"] == example_stock].sort_values("trading_date").copy()
)

# Calculate rolling averages for smoothing
stock_ts["sentiment_ma"] = (
    stock_ts["avg_sentiment"].rolling(window=20, center=True).mean()
)
stock_ts["return_ma"] = stock_ts["daily_return"].rolling(window=20, center=True).mean()

fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

# Plot 1: Sentiment over time
axes[0].plot(
    stock_ts["trading_date"],
    stock_ts["avg_sentiment"],
    alpha=0.3,
    color="blue",
    linewidth=0.5,
    label="Daily Sentiment",
)
axes[0].plot(
    stock_ts["trading_date"],
    stock_ts["sentiment_ma"],
    color="darkblue",
    linewidth=2,
    label="20-Day MA Sentiment",
)
axes[0].axhline(y=0, color="black", linestyle="--", linewidth=1, alpha=0.5)
axes[0].set_ylabel("Sentiment Score", fontsize=12, fontweight="bold")
axes[0].set_title(
    f"{example_stock}: Sentiment and Returns Time Series Analysis",
    fontsize=14,
    fontweight="bold",
)
axes[0].legend(loc="best")
axes[0].grid(True, alpha=0.3)

# Plot 2: Returns over time
axes[1].plot(
    stock_ts["trading_date"],
    stock_ts["daily_return"],
    alpha=0.3,
    color="green",
    linewidth=0.5,
    label="Daily Return",
)
axes[1].plot(
    stock_ts["trading_date"],
    stock_ts["return_ma"],
    color="darkgreen",
    linewidth=2,
    label="20-Day MA Return",
)
axes[1].axhline(y=0, color="black", linestyle="--", linewidth=1, alpha=0.5)
axes[1].set_xlabel("Date", fontsize=12, fontweight="bold")
axes[1].set_ylabel("Daily Return (%)", fontsize=12, fontweight="bold")
axes[1].legend(loc="best")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📊 Time Series Visualization for {example_stock}:")
print("   Compare visual patterns between sentiment (top) and returns (bottom)")
print("   Look for periods where sentiment and returns move together")

## 11. Statistical Significance Testing

Perform comprehensive statistical tests to validate correlation significance and calculate confidence intervals.

In [ ]:
# Statistical significance summary
print("🔬 Statistical Significance Analysis\n")
print("=" * 80)

# Overall significance
print(f"📊 OVERALL CORRELATION (All Stocks Combined):")
print(f"   Pearson r: {overall_corr:+.4f}")
print(f"   P-value: {overall_pval:.8f}")
print(f"   Sample size: {len(clean_data):,} observations")
print(f"   Significance level: α = 0.05")
print(
    f"   Result: {'✅ STATISTICALLY SIGNIFICANT' if overall_pval < 0.05 else '❌ NOT SIGNIFICANT'}"
)

# Calculate confidence interval for overall correlation
n = len(clean_data)
fisher_z = np.arctanh(overall_corr)
se = 1 / np.sqrt(n - 3)
ci_lower_z = fisher_z - 1.96 * se
ci_upper_z = fisher_z + 1.96 * se
ci_lower = np.tanh(ci_lower_z)
ci_upper = np.tanh(ci_upper_z)
print(f"   95% Confidence Interval: [{ci_lower:.4f}, {ci_upper:.4f}]")

print(f"\n" + "=" * 80)
print(f"📈 PER-STOCK SIGNIFICANCE TESTING:")
print(f"   Stocks tested: {len(correlations_df)}")
print(f"   Significant correlations (p < 0.05): {correlations_df['significant'].sum()}")
print(
    f"   Significance rate: {correlations_df['significant'].sum() / len(correlations_df) * 100:.1f}%"
)

# Power analysis (effect size)
print(f"\n" + "=" * 80)
print(f"🔍 EFFECT SIZE ANALYSIS (Cohen's Guidelines):")
print(f"   Small effect: |r| ≥ 0.10")
print(f"   Medium effect: |r| ≥ 0.30")
print(f"   Large effect: |r| ≥ 0.50")
print(f"\n   Overall correlation |r| = {abs(overall_corr):.4f}:")
if abs(overall_corr) >= 0.50:
    print(f"   ✅ LARGE effect size")
elif abs(overall_corr) >= 0.30:
    print(f"   ✅ MEDIUM effect size")
elif abs(overall_corr) >= 0.10:
    print(f"   ✅ SMALL effect size")
else:
    print(f"   ❌ Negligible effect size")

# Stock-level effect sizes
print(f"\n   Per-Stock Effect Sizes:")
for _, row in correlations_df.iterrows():
    abs_corr = abs(row["correlation"])
    if abs_corr >= 0.50:
        effect = "LARGE"
    elif abs_corr >= 0.30:
        effect = "MEDIUM"
    elif abs_corr >= 0.10:
        effect = "SMALL"
    else:
        effect = "Negligible"
    print(f"   {row['stock']}: |r|={abs_corr:.4f} → {effect}")

print(f"\n" + "=" * 80)
print(f"📋 INTERPRETATION SUMMARY:")
print(
    f"   • Correlation strength: {'Weak' if abs(overall_corr) < 0.3 else 'Moderate' if abs(overall_corr) < 0.5 else 'Strong'}"
)
print(
    f"   • Direction: {'Positive (sentiment ↑ → returns ↑)' if overall_corr > 0 else 'Negative (sentiment ↑ → returns ↓)'}"
)
print(
    f"   • Statistical validity: {'High confidence (p < 0.05)' if overall_pval < 0.05 else 'Low confidence (p ≥ 0.05)'}"
)
print(
    f"   • Practical significance: {'Actionable for trading strategies' if abs(overall_corr) >= 0.3 and overall_pval < 0.05 else 'Limited practical value'}"
)
print("=" * 80)

## 12. Investment Insights and Recommendations

Based on sentiment-return correlation analysis, develop actionable trading strategies and risk assessments for Nova Financial Solutions.

### Key Findings Summary

**Correlation Strength**: The analysis reveals [describe overall correlation] between news sentiment and stock returns across 6 major technology stocks.

**Statistical Significance**: [Discuss p-values and confidence intervals - whether relationships are statistically meaningful]

**Predictive Power**: Lagged correlation analysis shows [describe T+1, T+2 lag results] suggesting [predictive capability or lack thereof].

**Stock-Specific Patterns**: Individual stocks exhibit [describe variation in correlations] with [name stocks] showing strongest sentiment-return relationships.

### Trading Strategy Recommendations

1. **Sentiment-Based Entry Signals**:
   - For stocks with positive correlations (r > 0.30, p < 0.05): Consider long positions when daily sentiment exceeds +0.2
   - For stocks with negative correlations: Consider contrarian strategies

2. **Risk Management**:
   - Correlation strength varies by stock → diversification essential
   - Sentiment signals work best when combined with technical indicators
   - Stop-loss recommendations: [based on volatility analysis]

3. **Timing Considerations**:
   - Lagged correlations suggest [optimal entry timing based on T+1, T+2 results]
   - News volume matters: higher article counts → stronger signal reliability

### Limitations and Caveats

- **Correlation ≠ Causation**: Sentiment may reflect rather than predict price movements
- **Market Regime Dependency**: Results may vary in bull vs bear markets
- **Publisher Bias**: Consider news source concentration from Task 1 EDA
- **Sample Period**: Analysis covers [date range] - may not generalize to all market conditions

### Future Research Directions

- Incorporate technical indicators with sentiment for hybrid models
- Test intraday sentiment vs returns correlation
- Analyze sentiment-volatility relationships
- Develop machine learning models for return prediction using sentiment features

---

**Conclusion**: Sentiment analysis provides [weak/moderate/strong] predictive signals for stock returns. While statistically [significant/insignificant], the [small/medium/large] effect size suggests [actionable/limited] practical value for algorithmic trading strategies.